In [1]:
import pandas as pd
import re
import numpy as np

### Load Datasets

In [ ]:
dogs = pd.read_csv("dogs.csv")
#dogs.head()

In [3]:
travels = pd.read_csv("dogTravel.csv")
#travels.head()

In [4]:
population = pd.read_csv("NST-EST2021-POP.csv", names=['State', 'Pop'])
#population.head()

### Preprocessing

In [5]:
for index in dogs[(dogs['contact_country']!='US') & (dogs['contact_country']!='CA')].index:    #indici delle righe da traslare
    for i in range(1,8):                                                             #1-8 perchè sono 7 le colonne da traslare
        dogs.loc[index, dogs.keys()[33-i]]=dogs.loc[index, dogs.keys()[32-i]]   #prende il valore della colonna adiacente
    dogs.loc[index, 'status']=''                                               #non si può dire niente di status
    
#dogs.to_csv('dogs2.csv')

In [6]:
population['Pop']=population['Pop'].apply(lambda x: int(x.replace('.', '')))  
#trasformazione variabile Pop da str in int; è stato necessario rimuovere i punti di separazione

## 1. Extract all dogs with status that is not adoptable

In [ ]:
dogs[dogs['status']!='adoptable']

In [14]:
dogsna = dogs[dogs['status']!='adoptable']

## 2. For each (primary) breed, determine the number of dogs

In [ ]:
dogspb = dogs.groupby('breed_primary')['id'].count()
dogspb

## 3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. Hint: look at the secondary_breed.

In [20]:
dogssb = dogs[dogs['breed_secondary']=='Mixed Breed'].groupby('breed_primary')['id'].count()
#per ogni primary breed, calcolo di quanti cani hanno come valore 'Mixed Breed' per secondary breed

In [26]:
ratio = round(dogssb/dogspb*100, 3)
#ratio

In [28]:
for i in ratio.keys():         #per sostituire i NaN in 0
    if  not ratio[i] >=0:
        ratio[i]=0
#ratio

## 4. For each (primary) breed, determine the earliest and the latest posted timestamp.

In [29]:
l=[]   # eliminare dai valori in posted i caratteri 'T' e '+0000$' per facilitare la formattazione

for obj in dogs['posted']:
    obj=(re.sub('\+0000$', '', obj))
    res=(re.sub('T', ' ', obj))
    l.append(res)
    
dogs['posted_new']=l 

In [30]:
dogs['parsed posted'] = pd.to_datetime(dogs['posted'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
dogs.groupby('breed_primary')['parsed posted'].max() #the latest posted timestamp

In [ ]:
dogs.groupby('breed_primary')['parsed posted'].min() #the earliest posted timestamp

# # 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

In [34]:
dogsMale = dogs[dogs['sex']=='Male'].groupby('contact_state')['id'].count()

In [36]:
dogsFemale = dogs[dogs['sex']=='Female'].groupby('contact_state')['id'].count()

In [37]:
sexImbalance = dogsMale-dogsFemale

In [38]:
print("The sex imbalance is largest in " + str(sexImbalance.idxmax()) + ' with a value of: ' +str(sexImbalance.max()))

The sex imbalance is largest in OH with a value of: 205


## 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

In [ ]:
round(dogs.groupby(['age', 'size'])['stay_duration'].mean(),3)

In [ ]:
round(dogs.groupby(['age', 'size'])['stay_cost'].mean(),3)

## 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

In [43]:
df=travels.groupby('id').size().reset_index(name='number of travels') 
#nuovo dataframe, per ogni Id (per ogni cane) calcola il numero di viaggi effettuato

In [ ]:
df[df['number of travels']>=3].merge(dogs[['id', 'breed_primary']], on = 'id', how='left')

## 8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found.

In [49]:
manualToFound=travels['manual'].notnull() #indici delle righe dove dobbiamo copiare manual in found

In [50]:
foundToManual = travels['manual'].isnull() #indici delle righe dove dobbiamo copiare found in manual

In [51]:
travels.loc[manualToFound,'found']= np.nan #sostituiamo tutte le celle di found da modificare con NaN per poter usare 
                                            #funzione bfill
travels.head()

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,NaN,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,NaN,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


In [52]:
travels.loc[manualToFound, ['found', 'manual']]=travels[manualToFound][['found', 'manual']].bfill(axis=1)
#bfill sostiuisce ad un NaN il primo valore disponibile successivo. axis = 1 prende il valore sulla stessa riga(0 sarebbe colonna)

In [53]:
travels.loc[foundToManual, ['found', 'manual']]=travels[foundToManual][['found', 'manual']].ffill(axis=1)
#ffill sostituisce con primo valore disponibile precedente

## 9. For each state, compute the ratio between the number of travels and the population.

In [59]:
travels.loc[travels['contact_state']=='17325', 'contact_state']='PA' 
#alcune celle di contact_state avevano il valore 17325, cambiato con 'PA' perchè è uno zip code di pennsylvania e aveva PA
#come contact_city

In [60]:
state_dict = {
"Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA", "Colorado": "CO", "Connecticut": "CT",
"Delaware": "DE", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", 
"Iowa": "IA", "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD", "Massachusetts": "MA", 
"Michigan": "MI",  "Minnesota": "MN", "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV",
"New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY", "North Carolina": "NC", "North Dakota": "ND",
"Ohio": "OH", "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC", 
"South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT", "Vermont": "VT", "Virginia": "VA", "Washington": "WA",
"West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY", "District of Columbia": "DC",
}

In [61]:
for state in population.State:
    population.loc[population['State']==state, 'state_short'] = state_dict[state] #assegna valori della sigla

In [63]:
numTravels=travels.groupby('contact_state')['index'].count().to_frame() #crea dataframe per poter fare merge con altro dataframe
numTravels.columns=['number_of_travels'] #rinomina colonna
#numTravels 

In [65]:
travelsPop = pd.merge(population, numTravels, how='inner', left_on='state_short', right_on='contact_state')
#travelsPop

In [ ]:
travelsPop['number_of_travels']/travelsPop['Pop']*100 #calcola ratio, valori in percentuale

## 10. For each dog, compute the number of days from the posted day to the day of last access.

In [67]:
dogs['parsed accessed'] = pd.to_datetime(dogs['accessed'], format='%Y-%m-%d') #parse variabile accessed

In [72]:
difference= (dogs['parsed accessed']-dogs['parsed posted']) 
for key in difference.keys():
    number = difference[key].round('d').days # arrotonda per il numero di giorni e lo estrae 
    if number < 0:
        number = 0
    dogs.loc[key, 'days_difference']= number


In [74]:
#dogs.days_difference

## 11. Partition the dogs according to the number of weeks from the posted day to the day of last access.

In [77]:
dogs['weeks_difference'] = dogs['days_difference'] // 7
#dogs[['days_difference', 'weeks_difference']]

In [ ]:
dogs.groupby('weeks_difference')['id'].count()

## 12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.

In [81]:
stopwords=[]
f = open('stop_words_english.txt', 'r', encoding='utf-8')
for x in f:
    x=x.replace("'", "")
    x=x.replace('\n', "")
    stopwords.append(x)
f.close()
#stopwords

In [82]:
subset = pd.DataFrame(dogs[['breed_primary', 'sex','description', 'age', 'name']])
subset=subset.fillna(value='x')

In [83]:
breed_list = subset['breed_primary'].unique()
sex_list = subset['sex'].unique()
age_list = subset['age'].unique()

In [ ]:
listOfSimilar=[]
for breed in breed_list:
    print('\n new breed: ' + breed)
    for sex in sex_list:
        for age in age_list:
            idxs = list(subset.loc[(subset['breed_primary']==breed) & (subset['sex']==sex) & (subset['age']==age)].index.values)
            for idx in idxs:
                l=[]
                if subset.loc[idx,'description']!='x':
                    pos = idxs.index(idx)
                    listToCheck = ' '.join([word for word in subset.loc[idx, 'description'].split() if word not in stopwords]).split()
                    number = len(listToCheck)
                    if (not any(idx in sublist for sublist in listOfSimilar)):
                        for i in idxs[pos+1:]:
                            if subset.loc[idx,'name'] == subset.loc[i, 'name']:
                                count=0
                                listMatch = ' '.join([word for word in subset.loc[i, 'description'].split() if word not in stopwords]).split()
                                numberMatch = len(listMatch)
                                for w in listToCheck:
                                    if w in listMatch:
                                        count += 2
                                result = (count/(number + numberMatch))*100
                                if result >= 90 :
                                    l.append(i)
                    if len(l)>0:
                        l.append(idx)
                        listOfSimilar.append(l)

#print(listOfSimilar)

In [86]:
print(listOfSimilar)

In [87]:
dogs.loc[listOfSimilar[240]] 

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,accessed,type.y,description,stay_duration,stay_cost,posted_new,parsed posted,parsed accessed,days_difference,weeks_difference
21417,45361590,AL186,https://www.petfinder.com/dog/addison-b-453615...,Dog,Dog,Labrador Retriever,NaN,True,False,NaN,...,2019-09-20,Dog,SPAYED\n\nLAB MIX\n\nBORN 4-15-19\n\n15- 18 LB...,52,71.48,2019-07-25 15:17:48,2019-07-25 15:17:48,2019-09-20,56.0,8.0
17930,45361631,AL186,https://www.petfinder.com/dog/addison-b-453616...,Dog,Dog,Labrador Retriever,NaN,True,False,NaN,...,2019-09-20,Dog,*SPAYED\n\nLAB MIX\n\nBORN 4-15-19\n\n15- 18 L...,85,159.81,2019-07-25 15:20:42,2019-07-25 15:20:42,2019-09-20,56.0,8.0


In [88]:
print(subset.loc[21417,'description'] + '\nAAAAAAAAAAAAAAAAAAAA\n' + subset.loc[17930,'description'])

SPAYED

LAB MIX

BORN 4-15-19

15- 18 LBS

MEET ADDISON... SHE IS A CUTIE PIE, LOVING, AND YOUR NORMAL PUP. SHE IS GOOD WITH OTHER DOGS, KIDS AND CATS. SHE IS READY TO SHARE HER LIFE, AND LOVE WITH HER NEW FAMILY.  ADDISON IS FULLY VETTED AND READY TO GO!
TN POSTED 7/25/19

**It is rare that we know with certainty the ages or mixes that make up our wonderful dogs, but we do our best to be as accurate as possible based upon our many years in rescue.**

Adoption: $325 which covers quarantine, shots, worming, food, medical records, spaying/neutering, microchip,  and an Alabama State Health Certificate.

Transport, if needed :  $120.00    We consider the transport to be of great importance and, as such, take particular care of the dogs during the trip from Alabama.  We  make every effort to arrive with healthy and minimally stressed dogs.

Weight Scale:   Small =  less than 25
                       Medium = 26-60
                         Large =  61-100
                               XL =